# YAPAY SİNİR AĞLARI

In [2]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt 
# Standartlaştırma için eklenen fonksiyonlar scale ve StandartdScaler
from sklearn.preprocessing import scale 
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [3]:
df = pd.read_csv("verisetleri\Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.25, random_state=42)

In [4]:
# MODEL & TAHMİN

In [5]:
# Diğer bölümlerden farklı olarak değişkenlerde standartlaştırma işlemi gerçekleştirilecek. Makine algoritmalarının hemen hemen hepsi standartlaştırma işlemlerini normalde sever. Fakat bazı algoritmalar heterojen verisetleri üzerinde iyi çalışırken bazı algoritmalarda homojen verisetleri üzerinde iyi çalışır. Yapay sinir ağları homojen verisetleri üzerinde daha iyi çalışır. Çünkü hataların optimizasyonu noktasında yapılan türevleme işlemlerinde ortaya çıkmış olan aykırılıkların yani gerçek değerler ile tahmin değerleri arasında ciddi farklar oluşturan değerlerin ortaya çıkarabildiği bazı problemler sözkonusudur. Bundan dolayı yapay sinir ağlarını standartlaştırma işleri sonrasında kullanmak daha sağlıklı olmaktadır.

In [6]:
# STANDARTLAŞTIRMA

In [8]:
scaler = StandardScaler()

In [9]:
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [10]:
X_test_scaled = scaler.transform(X_test)

In [11]:
# mlp -> multi layer p... çok katmanlı algılayıcı
mlp_model = MLPRegressor().fit(X_train_scaled, y_train)

In [12]:
mlp_model

MLPRegressor()

In [13]:
# ( Burada gözükmüyor ) aktivasyon fonksiyonu 'relu'
# alpha = 0.0001, hidden_layer_sizes(100,), learning_rate_init = 0.001, solver='adam', max_iter=200 
# gibi değerler ön tanımlı değerlerdir.

In [14]:
# TAHMİN

In [15]:
mlp_model.predict(X_test_scaled)[0:5]

array([21.22999435, 39.19215033, 72.85521884, 15.22254478, 32.75666823])

In [16]:
y_pred = mlp_model.predict(X_test_scaled)

In [17]:
np.sqrt(mean_squared_error(y_test, y_pred))

656.2783247537075

In [18]:
# MODEL TUNNING

In [22]:
mlp_params = {"alpha": [0.1, 0.01, 0.02, 0.001, 0.0001], "hidden_layer_sizes" : [(10, 20), (5, 5), (100, 100)]}

In [20]:
# 0.001 ve 0.0001 değerleri daha küçük ve öğrenme işlemlerini yavaşlatıcı ama başarıyı arttırma ihtimali olduğunu varsaydığımız parametre değeri olarak verdik. alpha değerleri yani lambda değerleri yani ceza parametreleridir. 

In [21]:
# hidden_layer_sizes : Burada kullanmış olduğumuz fonksiyonlara kuruyor olduğumuz yapay sinir ağı modelinin topolojik yapısı ile ilgili bilgi vermek adına katmanları ifade etmek istediğimizde, katmanları ifade ederken şöyle bir yaklaşım izliyoruz. Girdiğimiz değer kadar katman ve girilen değerler kadar da hücre oluşmuş oluyor. 
# Yani burada iki tane gizli katman olması gerektiğini anlıyor. Biri 10 biri 20 sayılarını verdik. Herbir katmandaki nöron sayısını da bu sayılarla belirtik. Yani iki katman var, bir katmanda 10 nöron, diğer katmanda 20 nöron var. Bu birinci deneme içindir.
# İkinci deneme için yine iki gizli katman ve katmanlarda 5 er nöron belirledik.
# Başka bir deneme için 100 e 100 değerlerini verdik.

In [23]:
# Birinci alpha değeri için bütün gizli katman seçenekleri çalıştırılacak, sonra ikinci alpha değeri için bütün gizli katman seçenekleri çalıştırılacak. Bu şekilde devam edecek

In [36]:
mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv=10, verbose=2, n_jobs=-1).fit(X_train_scaled, y_train)

Fitting 10 folds for each of 15 candidates, totalling 150 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:   19.4s finished


In [37]:
mlp_cv_model.best_params_

{'alpha': 0.02, 'hidden_layer_sizes': (100, 100)}

In [27]:
# yapay sinir ağlarında kullanılabilecek olan bir çok ağ yapısı, kullanılabilecek olan bir çok farklı parametre değeri kullanılabilecek olan bir çok solver yöntemi yani çözüm yöntemi yine kullanılabilecek olan bir çok aktivasyon fonksiyonu yaklaşımı var. Dolayısıyla bu alan biraz daha uzmanlık gerektiren problemlerin türüne göre çalışmaların yapılabildiği bir alandır. 

In [28]:
# FİNAL MODELİ 

In [38]:
mlp_tunded = MLPRegressor(alpha=0.02, hidden_layer_sizes=(100,100)).fit(X_train_scaled, y_train)

In [39]:
y_pred = mlp_tunded.predict(X_test_scaled)

In [40]:
np.sqrt(mean_squared_error(y_test, y_pred))

368.5308375546997

In [41]:
# Olası faklı senaryolarda denenebilir.
# Değişkenler dönüştürülebilir.
# Başka yeni değişkenler oluşturulabilir.
# Henüz ön işlem olarak, eksik gözlem ve aykırı gözlem işlemlerini yapmadığımıza dikkat edelim. Bunlarda çok değerli hatayı düşürebilecek olan çalışmalardır. 

In [42]:
# the end nörons